In [ ]:
import polars as pl
import rtsvg
rt = rtsvg.RACETrack()
from  polars_spring_layout import PolarsSpringLayout
import random

edges_filename  = '../../data/stanford/facebook/348.edges'
_lu_  = {'fm':[], 'to':[]}
_pos_ = {}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
    _pos_[_fm_] = [10.0+random.random()*100.0, 20.0+random.random()*100.0]
    _pos_[_to_] = [10.0+random.random()*100.0, 20.0+random.random()*100.0]

df  = pl.DataFrame(_lu_)
_g_ = rt.createNetworkXGraph(df, [('fm','to')])

_psl_ = PolarsSpringLayout(_g_, _pos_)

In [ ]:
_df_  = _psl_.df_pos.sample(3)
_df2_ = _psl_.df_dist
_df_